<a href="https://colab.research.google.com/github/q890003/Hybrid-image_Image-pyramid_Colorizing-the-Russian-Empire/blob/master/Colorizing_the_Russian_Empire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time

def ncc(a,b):
    return np.sum(((a/(np.linalg.norm(a)**2)) * (b/(np.linalg.norm(b)**2))))

def nccAlign(a, b, t):
    max_ncc = -1
    ivalue=np.linspace(-t,t,2*t,dtype=int)
    jvalue=np.linspace(-t,t,2*t,dtype=int)
    for i in ivalue:
        for j in jvalue:
            nccDiff = ncc(a,np.roll(b,[i,j],axis=(0,1)))
            if nccDiff > max_ncc:
                max_ncc = nccDiff
                output = [i,j]
    return np.array(output)
def ssd(a,b):
    return np.sum(np.square(a-b))

def ssdAlign(a, b, t):
    max_ncc = -1
    ivalue=np.linspace(-t,t,2*t,dtype=int)
    jvalue=np.linspace(-t,t,2*t,dtype=int)
    for i in ivalue:
        for j in jvalue:
            nccDiff = ncc(a,np.roll(b,[i,j],axis=(0,1)))
            if nccDiff > max_ncc:
                max_ncc = nccDiff
                output = [i,j]
    return np.array(output)

img = cv2.imread('hw2_data/task3_colorizing/emir.tif',cv2.IMREAD_GRAYSCALE)
h, w = img.shape
shaved_img = img[int(h*0.01):int(h-h*0.02),int(w*0.05):int(w-w*0.05)]
h = shaved_img.shape[0] // 3
B = shaved_img[0:h,:]
G = shaved_img[h:2*h,:]
R = shaved_img[2*h:3*h,:]
h, w = B.shape

#B = B[h//2:,w//2:]
#G = G[h//2:,w//2:]
#R = R[h//2:,w//2:]

B_GP, G_GP, R_GP = [], [], []
B_GP.append(B)
G_GP.append(G)
R_GP.append(R)
while h > 128:
    B_GP.append(cv2.pyrDown(B_GP[-1]))
    G_GP.append(cv2.pyrDown(G_GP[-1]))
    R_GP.append(cv2.pyrDown(R_GP[-1]))
    h, w = B_GP[-1].shape
GtoB, RtoB = np.zeros(2,dtype=int), np.zeros(2,dtype=int)
i = len(G_GP)-1
while i >= 0:
    print("Processing pyramid level",i,end=', ')
    start = time.time()
    GtoB, RtoB = GtoB*2, RtoB*2
    G_GP[i] = np.roll(G_GP[i],GtoB,axis=(0,1))
    R_GP[i] = np.roll(R_GP[i],RtoB,axis=(0,1))
    if i < len(G_GP)-1:
        result = (np.dstack((R_GP[i],G_GP[i],B_GP[i]))).astype(np.uint8)
        plt.figure(figsize=(10,10)), plt.imshow(result)
        plt.title('Gaussian Pyramid level '+str(i)+' Result'), plt.axis('off')
    
    B_, G_, R_ = B_GP[i], G_GP[i], R_GP[i]
    alignGtoB = ssdAlign(B_,G_,3)
    alignRtoB = ssdAlign(B_,R_,3)
    GtoB += alignGtoB
    RtoB += alignRtoB
    i -= 1
    end = time.time()
    if i == len(G_GP)-1:
        G_GP[i] = np.roll(G_GP[i],GtoB,axis=(0,1))
        R_GP[i] = np.roll(R_GP[i],RtoB,axis=(0,1))
        result = (np.dstack((R_GP[i],G_GP[i],B_GP[i]))).astype(np.uint8)
        plt.figure(figsize=(10,10)), plt.imshow(result)
        plt.title('Gaussian Pyramid level '+str(i)+' Result'), plt.axis('off')
    print("time: %.4fs"%(end - start))
#h = shaved_img.shape[0] // 3
#B = shaved_img[0:h,:]
#G = shaved_img[h:2*h,:]
#R = shaved_img[2*h:3*h,:]
G = np.roll(G,GtoB,axis=(0,1))
R = np.roll(R,RtoB,axis=(0,1))
result = (np.dstack((R,G,B))).astype(np.uint8)
plt.figure(figsize=(10,10)), plt.imshow(result)
plt.title('Result'), plt.axis('off')